In [1]:
import pandas as pd
from sodapy import Socrata
from requests.exceptions import ReadTimeout
import time
from dotenv import load_dotenv
import os
import subprocess
import json

In [2]:
# Load the JSON data from the 'crime_data' key in the dictionary
crime_data_json = os.getenv('crime_data')
if crime_data_json:
    crime_data = json.loads(crime_data_json)
else:
    raise ValueError("crime_data not found in .env file or not in JSON format")

# Extract the required keys from the dictionary
keys_needed = ["app_token", "usr", "pw"]
credentials = {key: crime_data[key] for key in keys_needed}

# Connect to the Socrata client using the extracted credentials
client = Socrata('data.cityofnewyork.us',
                credentials["app_token"],
                username=credentials["usr"],
                password=credentials["pw"],
                timeout=30)  # Increase the timeout value to 30 seconds

# Define parameters
chunk = 2500
offset = 4000000
results = []

# Define retry logic parameters
max_retries = 5
retry_delay = 5  # seconds

while offset < 6000000:
    retries = 0
    while retries < max_retries:
        try:
            results_chunk = client.get("qgea-i56i", limit=chunk, offset=offset)
            break 
        except ReadTimeout:
            retries += 1
            print(f"Timeout occurred. Retrying... ({retries}/{max_retries})")
            time.sleep(retry_delay)
    else:
        print(f"Max retries exceeded. Unable to retrieve data.")
        break

    results.extend(results_chunk)
    
    offset += chunk

    if offset % (chunk * 20) == 0:
        print(f'Chunk created')

# Notification script
def play_sound_and_notify():
        for n in range(3):
            os.system('afplay /System/Library/Sounds/Hero.aiff')

        apple_script = f'display dialog "Next batch complete" buttons {{"OK"}}'
        subprocess.run(['osascript', '-e', apple_script])

play_sound_and_notify()

# Convert to DataFrame
df_1 = pd.DataFrame.from_records(results)

ValueError: crime_data not found in .env file or not in JSON format

In [10]:
import os
from dotenv import dotenv_values

secs = dotenv_values(".env")

crime_data_json = secs['crime_data']

print("Value of crime_data_json:", crime_data_json)

Python-dotenv could not parse statement starting at line 2
Python-dotenv could not parse statement starting at line 3


Value of crime_data_json: {"app_token": "mnjDHq54iove3iQDpGTd90eSw",


In [12]:
import os
import json

# Load environment variables from .env file
with open('.env', 'r') as f:
    env_content = f.read()

# Extract the value of the 'crime_data' variable from the .env content
crime_data_start = env_content.find('crime_data=') + len('crime_data=')
crime_data_end = env_content.find('\n', crime_data_start)
crime_data_json = env_content[crime_data_start:crime_data_end].strip()

# Parse the JSON string into a dictionary
crime_data_dict = json.loads(crime_data_json)

# Print the dictionary to verify
print("Parsed crime_data_dict:", crime_data_dict)


JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 43 (char 42)